# Dự Án Lớn Bắt Đầu!!

## Bộ Định Giá Sản Phẩm (The Product Pricer)

Một mô hình có khả năng ước tính giá trị của một sản phẩm dựa trên mô tả của nó.

## Quản Lý Dữ Liệu Phần 1

Hôm nay, chúng ta sẽ bắt đầu làm sạch và tuyển chọn bộ dữ liệu của mình bằng cách tập trung vào một phần nhỏ: Đồ Gia Dụng.

Bộ dữ liệu nằm ở đây:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

Và thư mục chứa tất cả các bộ dữ liệu sản phẩm ở đây:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw/meta_categories

In [ ]:
# imports

import os
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

## Thêm một lần import nữa - lớp Item

Nếu bạn gặp lỗi yêu cầu đồng ý với các điều khoản của Meta khi chạy đoạn này, hãy nhấp vào liên kết mà hệ thống cung cấp và làm theo hướng dẫn. Bạn sẽ được Meta phê duyệt trong vài phút.

Xem ô cuối cùng trong [colab này](https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing#scrollTo=FqyF5jZQkIl_) để biết các bước cần thực hiện nếu Meta không phê duyệt.

In [ ]:
from items import Item

In [ ]:
%matplotlib inline

In [ ]:
# Load in our dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_meta_Appliances", split="full", trust_remote_code=True)

In [ ]:
print(f"Number of Appliances: {len(dataset):,}")

In [ ]:
# Investigate a particular datapoint
datapoint = dataset[2]


In [ ]:
# Investigate

print(datapoint["title"])
print(datapoint["description"])
print(datapoint["features"])
print(datapoint["details"])
print(datapoint["price"])

In [ ]:
# How many have prices?

prices = 0
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            prices += 1
    except ValueError as e:
        pass

print(f"There are {prices:,} with prices which is {prices/len(dataset)*100:,.1f}%")

In [ ]:
# For those with prices, gather the price and the length

prices = []
lengths = []
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            prices.append(price)
            contents = datapoint["title"] + str(datapoint["description"]) + str(datapoint["features"]) + str(datapoint["details"])
            lengths.append(len(contents))
    except ValueError as e:
        pass

In [ ]:
# Plot the distribution of lengths

plt.figure(figsize=(15, 6))
plt.title(f"Lengths: Avg {sum(lengths)/len(lengths):,.0f} and highest {max(lengths):,}\n")
plt.xlabel('Length (chars)')
plt.ylabel('Count')
plt.hist(lengths, rwidth=0.7, color="lightblue", bins=range(0, 6000, 100))
plt.show()

In [ ]:
# Plot the distribution of prices

plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.2f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="orange", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# So what is this item??

for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 21000:
            print(datapoint['title'])
    except ValueError as e:
        pass

Cái này là cái gần nhất mình tìm được – có vẻ đang được bán với giá rất tốt!!

https://www.amazon.com/TurboChef-Electric-Countertop-Microwave-Convection/dp/B01D05U9NO/

## Bây giờ là lúc chọn lọc bộ dữ liệu của chúng ta

Chúng ta sẽ chọn các mục có giá từ 1 đến 999 USD.

Chúng ta sẽ tạo các đối tượng Item, trong đó văn bản sẽ được cắt ngắn để phù hợp trong 180 token bằng cách sử dụng Bộ tách token phù hợp.

Và sẽ tạo một prompt để sử dụng trong quá trình huấn luyện.

Những mục không đủ ký tự sẽ bị loại bỏ.

## Nhưng tại sao lại là 180 token??

Một câu rất hay – tại sao chúng ta lại cắt ngắn còn 180 token? Làm thế nào mà chúng ta xác định được con số đó?

Câu trả lời: đây là một ví dụ về “siêu tham số” (hyper-parameter). Nói cách khác, nó chủ yếu dựa trên việc thử nghiệm! Chúng ta muốn số lượng token đủ lớn để có đủ thông tin hữu ích nhằm đánh giá giá cả. Nhưng đồng thời cũng muốn giữ số lượng này thấp để huấn luyện hiệu quả hơn. Bạn sẽ thấy điều này ở Tuần 2.

Tôi bắt đầu với một con số có vẻ hợp lý, rồi thử nghiệm với một vài biến thể trước khi quyết định chọn 180. Nếu bạn có thời gian, bạn cũng nên làm như vậy! Bạn có thể tìm ra sự cân bằng tốt hơn và cải thiện kết quả. Việc thử và sai như thế này nghe có vẻ không thỏa mãn, nhưng lại là một phần quan trọng trong quá trình R&D khoa học dữ liệu.

Có một lý do thú vị khác khiến chúng ta nên ưu tiên số lượng token thấp hơn trong dữ liệu huấn luyện. Khi cuối cùng chúng ta sử dụng mô hình để dự đoán, chúng ta sẽ cung cấp các sản phẩm mới và yêu cầu mô hình 


In [ ]:
# Create an Item object for each with a price

items = []
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            item = Item(datapoint, price)
            if item.include:
                items.append(item)
    except ValueError as e:
        pass

print(f"There are {len(items):,} items")

In [ ]:
# Look at the first item

items[1]

In [ ]:
# Investigate the prompt that will be used during training - the model learns to complete this

print(items[100].prompt)

In [ ]:
# Investigate the prompt that will be used during testing - the model has to complete this

print(items[100].test_prompt())

In [ ]:
# Plot the distribution of token counts

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="green", bins=range(0, 300, 10))
plt.show()

In [ ]:
# Plot the distribution of prices

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="purple", bins=range(0, 300, 10))
plt.show()

## Lưu ý nhỏ

Nếu bạn thích sự đa dạng màu sắc mà matplotlib có thể sử dụng trong các biểu đồ, bạn nên đánh dấu trang này lại:

https://matplotlib.org/stable/gallery/color/named_colors.html

## Việc cần làm cho bạn:

- Xem lại lớp Item và đảm bảo bạn đã hiểu rõ về nó
- Kiểm tra một số đối tượng Item, xem prompt huấn luyện với `item.prompt` và prompt kiểm tra với `item.test_prompt()`
- Tạo thêm một số biểu đồ histogram để hiểu rõ hơn về dữ liệu

## Lần tới chúng ta sẽ kết hợp với nhiều loại sản phẩm khác

Như Electronics và Automotive. Điều này sẽ mang lại cho chúng ta một bộ dữ liệu khổng lồ, và lúc đó chúng ta có thể chọn lọc kỹ lưỡng để lấy ra tập con phù hợp nhất cho việc huấn luyện.